# AirBnB assignment problem

You are planning a weekend vacation in Boston for a large group of students. You would like to stay in AirBnB's, since this is generally cheaper than staying at an hotel.
Unfortunately, this is quite complicated to organize!

* AirBnBs accommodate different numbers of guests, so it is difficult to divide everyone into groups
* Since there are many of you, some people might try to book the same listings and be disappointed
* All the AirBnBs have different features and prices

Instead of everyone trying to organize their own groups and bookings, you have volunteered to formulate an optimization model to determine where each person should stay.
You sent out an initial survey to your classmates to find out what preferences and constraints you need to consider:
* Some students requested to be placed in a Male only/Female only lisitng, while some were happy to share with anyone
* Some students requested specific amenities: Kitchen, Air-con etc.

You also decided to restrict the listings that you would consider based on certain criteria
* You all want to be near each other for group activities, so we will only consider listings in certain neighbourhoods near central Boston or Back Bay
* We will only consider listings with high review scores

Before you begin, check your current working directory. If this is not inside the folder "3_optimization", change it to this location using the `cd()` function.

Note: for Windows paths, use "\\". For UNIX systems, use "/".

In [ ]:
pwd()

In [ ]:
# set the working directory to the 3_optimization folder
#cd("C:\\Users\\Name\\Desktop\\mban_orientation\\1_orientation\\3_optimization\\")
#cd("/Users/Name/Desktop/mban_orientation/1_orientation/3_optimization/")

In [3]:
# load packages
using JuMP, Gurobi, DataFrames, CSV

## Data

Yesterday we practiced filtering and manipulating the AirBnB data in R. Instead of starting from scratch in Julia, let's use our R skills to do some initial data processing.
To do this, we'll need to do the following:
* an R script that contains all the code we want to run: input_data.R
* a way to tell Julia to run the R script
* a way to read the output files from the R script into Julia (we'll save the output as a csv file in R, then read this file in Julia)

How do we do the middle step? 
UNIX:
* the ``run( [command] )`` function tells Julia to send a command to the terminal
* the `Rscript [script] [arguments]` command tells your computer to open an R session and run a specific script

Windows:
* the ``run( [command] )`` function tells Julia to send a command to the terminal
* the `Rscript.exe [script] [arguments]` command tells your computer to open an R session and run a specific script. You will need to figure out where the file "Rscrip.exe" is located on your computer and provide the path to this file.

Note: if this doesn't work on your computer, open the R script in RStudio. Set the `date` variable in the script and run the code manually.


In [4]:
# our R script takes one argument: the date that we will start our stay
# set the date that we want to start our booking
date = "2019-11-25"

# this line will run the R script 
#run(`Rscript input_data.R $date`)
#run(`"C:\Program Files\R\R-3.6.1\bin\Rscript.exe" input_data.R $date`)

"2019-11-25"

In [5]:
# now, let's read in the output that we saved from this script 
listings = CSV.read("filtered_listings.csv")
L=size(listings,1)
println("There are $L listings available on $date")
first(listings,5)

There are 61 listings available on 2019-11-25


,id,listing_url,scrape_id,last_scraped,name
,Int64,String,Int64,Dates…,String
1,820073,https://www.airbnb.com/rooms/820073,20190714024644,2019-07-14,"Modern Loft, 1700 SqFt. Location!"
2,1868124,https://www.airbnb.com/rooms/1868124,20190714024644,2019-07-14,Lux Downtown Boston 1 Bedroom Apt w/pool
3,1868513,https://www.airbnb.com/rooms/1868513,20190714024644,2019-07-14,Lux 1 Bedroom in Post-War Back Bay building w/WiFi
4,1966195,https://www.airbnb.com/rooms/1966195,20190714024644,2019-07-14,Lux Downtown Boston 2 Bedroom Apt w/pool
5,2187766,https://www.airbnb.com/rooms/2187766,20190714024644,2019-07-14,Lux Downtown Boston 2 Bedroom Apt w/pool


The file that we have read into Julia contains all the listings that are available for a 3-night stay starting on our specified date. The columns of this DataFrame include all the original AirBnB data, plus a few new columns that we added in the R script. For example, we added new columns to indicate whether each listing has certain amenities.

In [6]:
# Let's check if our amenity columns have been added
amenities = [:Kitchen,:Air_conditioning]
listings[!,amenities]

,Kitchen,Air_conditioning
,Int64,Int64
1,1,1
2,1,1
3,1,1
4,1,1
5,1,1
6,1,1
7,1,1
8,1,1
9,1,1


In [7]:
# select the column in the listings data that gives the cost on this date
cost = listings[!,Symbol("stay 1")]

61-element CSV.Column{Float64,Float64}:
 1566.0
 1266.0
 1335.0
 1749.0
 1749.0
 1335.0
  810.0
  882.0
  975.0
  800.0
 1749.0
 2250.0
 1084.0
    ⋮  
  723.0
  615.0
 1056.0
  675.0
  510.0
 1197.0
  994.0
  663.0
  750.0
  181.0
  195.0
  225.0

Now let's read in the date we gathered about students' accommodation preferences.

In [8]:
preferences = CSV.read("preferences.csv")
N=size(preferences,1)
first(preferences,5)

,Name,room_F,room_M,room_A,Air_conditioning,Kitchen,Gym
,String,Int64,Int64,Int64,Int64,Int64,Int64
1,John Nicholas,0,1,1,1,1,1
2,Michael Rudahl,0,1,0,1,1,0
3,All A-Sloan,1,0,1,1,0,0
4,JM,1,0,0,1,1,0
5,Joey Khoury El Aramouni,0,0,1,1,0,0


## Model formulaton

Now that we have all our data, let's formulate a model to select the cheapest set of listings that we could book to accommodate everyone.
* We need to decide which listings to book. We'll need a **binary** variable to represent whether we should book each listing: $$y_j =\begin{cases} 1 & \text{if we book listing $j$} \\ 0& \text{if we don't book listing $j$}\end{cases}$$
* We need to decide who will stay in each listing that we book: $$x_{ij} =\begin{cases} 1 & \text{if person $i$ stays in listing $j$} \\ 0& \text{if not}\end{cases}$$
* We need a cost function to calculate the total price of our bookings: $$\sum_{j=1}^L cost_j y_j$$

We also need some constraints to make sure our solution is feasible:

$ \sum_{j=1}^{L} x_{ij} = 1 \text{ for } i=1,\dots,N$ (each person is assigned to exactly one listing)

$ x_{ij} \leq y_j \text{ for } i=1,\dots,N; \ j=1,\dots,L $ (can only assign people to listings that we book)

$ \sum_{i=1}^{N} x_{ij} \leq \text{accommodates}_j \text{ for } j=1,\dots,L$ (we can't assign more people than the listing allows)

In [10]:
# Let's try a simple model to select the cheapest listings that are available for this period
m = Model(with_optimizer(Gurobi.Optimizer,TimeLimit=60))

# variables
# y[j]=1 listing j is booked, and 0 otherwise
@variable(m, y[1:L], Bin)
# x[i,j]=1 if person i stays in listing j, and 0 otherwise
@variable(m, x[1:N, 1:L], Bin)

# constraints
# everyone has to be assigned exactly one listing
@constraint(m, [i=1:N], sum(x[i,:])==1) 
# we can only put people in a listing if it is booked
@constraint(m, [i=1:N, j=1:L], x[i,j]<=y[j]) 
# maximum occupancy per unit -- we multiply this by y to make the problem easier to solve
@constraint(m, [j=1:L], sum(x[:,j])<=listings[j,:accommodates])

# objective
@objective(m, Min, sum(cost[j]*y[j] for j=1:L))

optimize!(m)

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Optimize a model with 3905 rows, 3843 columns and 15128 nonzeros
Variable types: 0 continuous, 3843 integer (3843 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 3e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 7e+00]
Found heuristic solution: objective 36294.000000
Presolve removed 248 rows and 0 columns
Presolve time: 0.04s
Presolved: 3657 rows, 3843 columns, 14636 nonzeros
Variable types: 0 continuous, 3843 integer (3843 binary)

Root relaxation: objective 6.811613e+02, 3817 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  681.16129    0 1575 36294.0000  681.16129  98.1%     -    0s
H    0     0                    36184.000000  681.16129  98.1%     -    0s
H    0     0                    

In [11]:
# let's look at our solution:
println("Cost per person: ", JuMP.objective_value(m)/N)
println("Number of listings booked: ", sum(JuMP.value.(y)))
println("Selected listings: ", findall(JuMP.value.(y).>0) )
println("Guests per listing: ", listings[findall(JuMP.value.(y).>0),:accommodates] )

Cost per person: 198.25806451612902
Number of listings booked: 19.0
Selected listings: [8, 14, 16, 22, 23, 30, 33, 34, 36, 38, 39, 49, 50, 52, 56, 57, 59, 60, 61]
Guests per listing: [5, 4, 3, 3, 2, 4, 4, 4, 2, 4, 2, 4, 3, 6, 5, 4, 1, 1, 1]


Now, let's incorporate some of the other constraints from our preference data.

We need new variables to decide whether each listing should be booked for males only, females only, or any gender. For convenience, let's index these variables with the column names in our preferences data.
$$ z_{j, k} = \begin{cases} 1 & \text{ if listing $j$ is used as room type $k$} \\ 0 & \text{ otherwise}.\end{cases}$$
Next, we need a constraint to ensure that each listing is only used for one of the three categories (i.e., a listing can't be male only and female only at the same time).
$$ \sum_{k \in \text{ room types} } z_{jk} \leq 1 \text{ for } j=1,\dots,L$$

We also need to ensure that each student is placed in a room type that matches their preferences:
$$ x_{ij} \leq \sum_{k \in \text{ room types} } z_{jk}\text{preferences}_{ik} \quad \text{ for } i=1,\dots,N; \ j=1,\dots,L $$

In [12]:
room_types = [:room_A ,:room_F ,:room_M ]
@variable(m, z[1:L,room_types], Bin)

# each listing can only be used for one of the three room types
@constraint(m, [j=1:L], sum(z[j,:]) <= 1)

# preferences: x[i,j] can only be 1 if listing j matches the student's preferred room type(s) 
# i.e., z[j,type] matches student i's gender preferences
@constraint(m, [i=1:N,j=1:L], x[i,j] <= sum(preferences[i,k]*z[j,k]  for k in room_types))

optimize!(m)


Optimize a model with 7748 rows, 4026 columns and 25315 nonzeros
Variable types: 0 continuous, 4026 integer (4026 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 3e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 7e+00]

MIP start did not produce a new incumbent solution

Found heuristic solution: objective 38275.000000
Presolve removed 659 rows and 8 columns
Presolve time: 0.10s
Presolved: 7089 rows, 4018 columns, 21785 nonzeros
Variable types: 0 continuous, 4018 integer (4018 binary)

Root relaxation: objective 6.811613e+02, 4444 iterations, 0.09 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  681.16129    0 1650 38275.0000  681.16129  98.2%     -    0s
H    0     0                    38261.000000  681.16129  98.2%     -    0s
H    0     0                    16869.000000  681.16129  96.0%     -    0

In [13]:
# let's look at our solution:
println("Cost per person: ", JuMP.objective_value(m)/N)
println("Number of listings booked: ", sum(JuMP.value.(y)))
println("Selected listings: ", findall(JuMP.value.(y).>0) )
println("Guests per listing: ", listings[findall(JuMP.value.(y).>0),:accommodates] )

Cost per person: 198.25806451612902
Number of listings booked: 19.0
Selected listings: [8, 14, 16, 22, 23, 30, 33, 34, 36, 38, 39, 49, 50, 52, 56, 57, 59, 60, 61]
Guests per listing: [5, 4, 3, 3, 2, 4, 4, 4, 2, 4, 2, 4, 3, 6, 5, 4, 1, 1, 1]


Now let's add a constraint for the amenities requested by each student in the preferences data (in these columns, 1 means that the student must be in a listing with this amenity, 0 means they don't mind).
How can we express this as an optimization constraint? Ideally we would like something that looks like this:

$$x_{ij}\leq \text{amenities_match}(i,j),$$

where amenities_match(i,j) is a function that returns 1 if listing $j$ has all the amentities requested by student $i$, and zero if it does not. We could write this function using a for loop to check each amenity and if statements to return either 1 or 0.

In [14]:
function amenities_match(i,j)
    student_preferences = preferences[i,amenities]
    listing_amenities = listings[j,amenities]
    for a in amenities
        if student_preferences[a]>listing_amenities[a]
            # listing lacks the requested feature
            return(0)
        end
    end
    #all amenities matched student's preferences
    return(1)
end

amenities_match (generic function with 1 method)

In [15]:
m = Model(with_optimizer(Gurobi.Optimizer,TimeLimit=60))

# variables
# y[j]=1 listing j is booked, and 0 otherwise
@variable(m, y[1:L], Bin)
# x[i,j]=1 if person i stays in listing j, and 0 otherwise
@variable(m, x[1:N,1:L], Bin)

# constraints
# everyone has to be assigned exactly one listing
@constraint(m,[i=1:N],sum(x[i,:])==1) 
# we can only put people in a listing if it is booked
@constraint(m,[i=1:N,j=1:L],x[i,j]<=y[j]) 
# maximum occupancy per unit
@constraint(m,[j=1:L],sum(x[:,j])<=listings[j,:accommodates]*y[j]) 

# objective
@objective(m, Min, sum( y[j]*cost[j] for j=1:L ))# variables

@variable(m, z[1:L,room_types], Bin)

# each listing can only be used for one of the three room types
@constraint(m, [j=1:L], sum(z[j,:]) <= 1)

# preferences: x[i,j] can only be 1 if listing j matches the student's preferred room type(s) 
# i.e., z[j,type] matches student i's gender preferences
@constraint(m, [i=1:N,j=1:L], x[i,j] <= sum(preferences[i,k]*z[j,k]  for k in room_types))

# amenity preferences: x[i,j] can only be 1 if listing[j] matches student i's preferences
@constraint(m,[i=1:N,j=1:L], x[i,j] <= amenities_match(i,j) )

optimize!(m)

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Optimize a model with 11530 rows, 4026 columns and 29158 nonzeros
Variable types: 0 continuous, 4026 integer (4026 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [2e+02, 3e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 34528.000000
Presolve removed 5643 rows and 782 columns
Presolve time: 0.13s
Presolved: 5887 rows, 3244 columns, 17796 nonzeros
Variable types: 0 continuous, 3244 integer (3244 binary)

Root relaxation: objective 1.229200e+04, 6018 iterations, 0.38 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    12292.000000 12292.0000  0.00%     -    0s

Explored 0 nodes (7504 simplex iterations) in 0.56 seconds
Thread count was 4 (of 4 available processo

In [16]:
println("Cost per person: ",JuMP.objective_value(m)/N)
println("Number of listings: ",sum(JuMP.value.(y)))
println("Selected listings: ",findall(JuMP.value.(y).>0) )
println("Guests per listing: ",listings[findall(JuMP.value.(y).>0),:accommodates] )

Cost per person: 198.25806451612902
Number of listings: 19.0
Selected listings: [8, 14, 16, 22, 23, 30, 33, 34, 36, 38, 39, 49, 50, 52, 56, 57, 59, 60, 61]
Guests per listing: [5, 4, 3, 3, 2, 4, 4, 4, 2, 4, 2, 4, 3, 6, 5, 4, 1, 1, 1]


In [17]:
# find out what type of room each listing will be
listings[:,:type] = ""
Z = JuMP.value.(z)

for j in findall(JuMP.value.(y).>0)
    for t in room_types
        if Z[j,t]==1
         listings[j,:type] = string(t)
        end
    end
end
listings[!,:type]

┌ Warning: implicit vector broadcasting in setindex! is deprecated; use `df[:, col_inds] .= Ref(v)` broadcasting assignment to change the columns in place
│   caller = top-level scope at In[17]:1
└ @ Core In[17]:1
┌ Warning: Implicit broadcasting to a new column in DataFrame assignment is deprecated.Use `df[!, col_ind] .= v`  when `df` has some columns
│   caller = setindex!(::DataFrame, ::String, ::Colon, ::Symbol) at deprecated.jl:1584
└ @ DataFrames /Users/EZ/.julia/packages/DataFrames/XuYBH/src/deprecated.jl:1584


61-element Array{String,1}:
 ""      
 ""      
 ""      
 ""      
 ""      
 ""      
 ""      
 "room_F"
 ""      
 ""      
 ""      
 ""      
 ""      
 ⋮       
 "room_A"
 ""      
 "room_F"
 ""      
 ""      
 ""      
 "room_A"
 "room_A"
 ""      
 "room_A"
 "room_A"
 "room_M"

In [18]:
X = JuMP.value.(x)

assigned_listing = [ listings[findall(X[i,:].>0)[1],:id] for i=1:N]
assigned_type = [ listings[findall(X[i,:].>0)[1],:type] for i=1:N]

62-element Array{String,1}:
 "room_M"
 "room_M"
 "room_A"
 "room_F"
 "room_A"
 "room_M"
 "room_A"
 "room_A"
 "room_F"
 "room_A"
 "room_A"
 "room_M"
 "room_A"
 ⋮       
 "room_A"
 "room_A"
 "room_A"
 "room_A"
 "room_A"
 "room_F"
 "room_A"
 "room_F"
 "room_A"
 "room_A"
 "room_A"
 "room_M"

In [19]:
# make a new data frame with each individual's assignment
assignments = DataFrame()
assignments[!,:Name] = preferences[!,:Name]
assignments[!,:listing] = assigned_listing
assignments[!,:type] = assigned_type
first(assignments,5)

,Name,listing,type
,String,Int64,String
1,John Nicholas,14994014,room_M
2,Michael Rudahl,14994014,room_M
3,All A-Sloan,23871901,room_A
4,JM,32831210,room_F
5,Joey Khoury El Aramouni,21547937,room_A


In [20]:
# choose a name for the output file
filename = "airbnb_solution"

CSV.write("$filename.csv",assignments)

"airbnb_solution.csv"

Great! We now have our initial solution, which will be significanlty cheaper than staying in an hotel. 
The next step is to validate our solution:
* each individual student should find their listing and make sure it matches their preferences
* each student should check their roommates and make sure that they are in the correct type of room
* are there any problems/complaints about the assignments?


It's generally cheaper for many people to share a listing, so our optimization model seems to be choosing listings that accommodate many people. Unfortunately, some hosts might be a little optimistic about the number of people that their space can hold!
Let's do a quick comparison of the number of beds vs. number of guests in our selected listings.

In [ ]:
selected_listings = findall(JuMP.value.(y).>0)
listings[selected_listings,[:id,:beds,:accommodates]]

It looks like most listings are assuming two people per bed. Some even have more than two people per bed --- maybe there is a spare mattress or couch?

Let's see what happens if we restrict the number of people assigned to each listing to the number of beds in the listing:

In [ ]:
# maximum occupancy per unit
@constraint() 
optimize!(m)

In [ ]:
println("Cost per person: ",JuMP.objective_value(m)/N)
println("Number of listings: ",sum(JuMP.value.(y)))

This solution is significantly more expensive. What should we do?

# Exercise

Divide into groups to work on an improved version of the AirBnB assignment model. There is no "right" solution to this problem, so your goal is to come up with the best model that you can. After working on this, you will have an opportunity to present it to other students and get their feedback.
Here are some ideas to get you started:
* You can modify the R script to generate different input data for your model (e.g., you could add different neighbourhoods, change the start date of the trip, adjust the rating requirements). If you do this, keep a list of your criteria and the reason for these decisions.
* You can ask your classmates for additional data about their preferences. Remember that collecting data is difficult and time-consuming, so you can't just assume that everyone will answer a 10-page questionnaire with detailed questions about their preferences. You need to justify why it's worthwhile to put effort into collecting more data. If you decide to do this, add the corresponding columns in the preferences.csv file and type in some made-up data so that you can run your model (unfortunately we don't have time for everyone to fill out more surveys).
* Think about how you would present your solution to your classmates. Maybe you could send them a dashboard that shows the details of their assignment and roommates. Perhaps you could visualize all the selected bookings on a map so that everyone can see where their friends will be located.

Outputs
* Provide a brief summary of how and why you have modified the assignment model. Describe any assumptions you have made and any additional data that you are using.
* Provide a notebook with a full version of your code. Use comments to explain any new constraints or variables that you have added.
* Provide a csv file with your best solution and an example of how you would share your solution with other students (plots, dashboards etc.)

In [50]:
m = Model(with_optimizer(Gurobi.Optimizer,TimeLimit=60))

cost = listings[!,Symbol("stay 1")]
rating = listings[!,:review_scores_rating]

# variables
# y[j]=1 listing j is booked, and 0 otherwise
@variable(m, y[1:L], Bin)
# x[i,j]=1 if person i stays in listing j, and 0 otherwise
@variable(m, x[1:N,1:L], Bin)

# constraints
# everyone has to be assigned exactly one listing
@constraint(m,[i=1:N],sum(x[i,:])==1) 
# we can only put people in a listing if it is booked
@constraint(m,[i=1:N,j=1:L],x[i,j]<=y[j]) 
# maximum occupancy per unit
# @constraint(m,[j=1:L],sum(x[:,j])<=listings[j,:accommodates]*y[j]) 

@constraint(m, sum(x[:,:]) == N)

@constraint(m, [j=1:L], sum(x[:,j]) == listings[j,:accommodates]*y[j])

# objective
@objective(m, Max, sum( y[j]*(rating[j]/(cost[j] / listings[j,:accommodates])) for j=1:L ))# variables

@variable(m, z[1:L,room_types], Bin)

# each listing can only be used for one of the three room types
@constraint(m, [j=1:L], sum(z[j,:]) <= 1)

# preferences: x[i,j] can only be 1 if listing j matches the student's preferred room type(s) 
# i.e., z[j,type] matches student i's gender preferences
@constraint(m, [i=1:N,j=1:L], x[i,j] <= sum(preferences[i,k]*z[j,k]  for k in room_types))

# amenity preferences: x[i,j] can only be 1 if listing[j] matches student i's preferences
@constraint(m,[i=1:N,j=1:L], x[i,j] <= amenities_match(i,j) )

optimize!(m)

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Optimize a model with 11531 rows, 4026 columns and 32940 nonzeros
Variable types: 0 continuous, 4026 integer (4026 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [2e-01, 5e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 6e+01]
Presolve removed 5644 rows and 782 columns
Presolve time: 0.10s
Presolved: 5887 rows, 3244 columns, 17788 nonzeros
Variable types: 0 continuous, 3244 integer (3244 binary)

Root relaxation: objective 9.951142e+00, 1146 iterations, 0.04 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    9.95114    0   12          -    9.95114      -     -    0s
     0     0    9.94614    0   11          -    9.94614      -     -    0s
H    0     0                       9.9044858    9.94614  0.42%     -    0s
  

In [59]:
println("Cost per person: ",JuMP.objective_value(m)/N)
println("Number of listings: ",sum(JuMP.value.(y)))
println("Selected listings: ",findall(JuMP.value.(y).>0) )
println("Guests per listing: ",listings[findall(JuMP.value.(y).>0),:accommodates] )

out = listings[findall(JuMP.value.(y).>0),[:id, :name, :accommodates, :longitude, :latitude, :neighbourhood]]

Cost per person: 0.16015914662355393
Number of listings: 27.0
Selected listings: [10, 14, 15, 18, 20, 21, 22, 23, 24, 27, 30, 32, 36, 37, 38, 39, 40, 46, 49, 50, 51, 54, 57, 58, 59, 60, 61]
Guests per listing: [2, 4, 2, 2, 2, 2, 3, 2, 2, 2, 4, 2, 2, 2, 4, 2, 2, 2, 4, 3, 2, 1, 4, 2, 1, 1, 1]


,id,name,accommodates,longitude
,Int64,String,Int64,Float64
1,11201204,"Stylish Back Bay Studio, Steps to Newbury & Hynes",2,-71.0876
2,14994014,"Downtown!!Stay in a modern apt, walk everywhere!",4,-71.0642
3,15127120,Sonder | Winthrop Square | Sleek Studio + Laundry,2,-71.0593
4,15338096,Sonder | Winthrop Square | Airy Studio + Laundry,2,-71.0598
5,15581608,New Stylish Studio Overlooking Newbury Street!,2,-71.0853
6,15582062,Cozy Studio on Enchanting Newbury Street!,2,-71.0841
7,17573922,"NEW 1st Floor, 1BR Downtown Boston",3,-71.0616
8,19084632,Private Room in Back Bay for the Summer!,2,-71.0871
9,20635474,(104) Back Bay Studio - Prime Location/ St Botolph,2,-71.082


In [46]:
termination_status(m)

INFEASIBLE_OR_UNBOUNDED::TerminationStatusCode = 6

In [60]:
filename = "coordinates of top ratios"

CSV.write("$filename.csv",out)

"coordinates of top ratios.csv"